In [1]:
import geemap, ee

In [2]:
#Uncomment this when running for the first time and/or if the map doesnt render
# ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [3]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

This bit is needed if you're making changes to the qm lib file and want to reload it live

In [4]:
%reload_ext autoreload
import quality_mosaic as qm

# Choose date and location

## Define parameters

In [5]:
lat, lon = 33.869066751,-118.405718865
date = '2021-01-18'

## Define Hyperparameters

In [6]:
#define the total number of days before and after the date of interest you want to search``
max_search_window_months = 6
# define the number of days to median filtered S1 images
s1_median_days = 6

Convert to ee objects

In [7]:
point = ee.Geometry.Point([lon, lat])
sample = point.buffer(10).bounds()
roi = point.buffer(100).bounds()
sample

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.bounds",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Geometry.buffer",
          "arguments": {
            "distance": {
              "constantValue": 10
            },
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Point",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      -118.405718865,
                      33.869066751
                    ]
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [8]:
visParamsRGB = {"min": 0, "max": 4000, "bands": ["B4", "B3", "B2"]}
visParamsVV  = {"min": -30, "max": 0, "bands": ["VV"]}

visParamsFScore  = {"min": -50, "max": 0, "bands": ["FScore"]}

green_blue_yellow_orange = ['#008080','#0039e6','#FFFF31','#f85d31']
visParamsMTGSI  = {'min':-0.5, 'max':0.25, 'palette':green_blue_yellow_orange, 'bands':['mTGSI']}

# Get mosaics for S1 and S2

In [9]:
s1_s2 = qm.get_s1_s2(roi=roi, date=date, max_search_window_months=6,s1_median_days=s1_median_days)
s1_s2.bandNames().getInfo()

Search window from 2020-07-18 to 2021-07-18


['B2',
 'B3',
 'B4',
 'B8',
 'B8A',
 'B11',
 'B12',
 'VV',
 'VH',
 'mTGSI',
 'BSI',
 'NDWI']

# Show interactive map

Add a marker manually if you want to change the location of the sample pointm

In [13]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(s1_s2, visParamsRGB, 'qm_s2')
Map.addLayer(s1_s2, visParamsVV, 's1')
Map.addLayer(s1_s2,visParamsMTGSI , 'mTGSI')
#Add the buffer around sampled point
Map.addLayer(sample,
             {'color':'green', 'opacity':0.4},
             name='buffer')

Map.add_marker([lat, lon],tooltip='sample', name='sample', draggable=False)
Map.centerObject(sample, 20)

Map

Map(center=[33.869066767472795, -118.40571869467622], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
# Map = geemap.Map()
# left_layer = geemap.ee_tile_layer(s1_s2, visParamsRGB, 'S2')
# right_layer = geemap.ee_tile_layer(s1_s2, visParamsMTGSI , 'mTGSI')

# # Map.addLayer(s1_s2, visParamsMTGSI , 'mTGSI')
# # Map.addLayer(s1_s2, visParamsVV, 's1', False)
# # Map.addLayer(s1_s2, visParamsRGB, 'qm_s2')

# Map.centerObject(sample, 18)
# Map.add_basemap('SATELLITE')
# Map.split_map(left_layer, right_layer)

# #Add the buffer around sampled point
# Map.addLayer(sample,
#             {'color':'green', 'opacity':0.4},
#             name='buffer')

# Map.add_marker([lat, lon],tooltip='sample', name='sample', draggable=False)
# Map

### If you dropped a new pin, make sure to run this cell:

In [18]:
if (Map.draw_last_feature is not None):
    
    Map.draw_last_feature.geometry().getInfo()['coordinates']
    new_sample = Map.draw_last_feature.buffer(10).bounds()
    Map.addLayer(new_sample,
             {'color':'green', 'opacity':0.3},
             name='new_buffer')
    print("New Sampled Point: ", Map.draw_last_feature.geometry().getInfo()['coordinates'])

### If you're happy with the location of the *most recent* pin, then run this cell:

In [33]:
#check if new_sample is defined
try:
    sample = new_sample.geometry()
    #print the coordinates of the new pin
    print(Map.draw_last_feature.geometry().getInfo()['coordinates'])
except NameError:
    print ("You didnt drop a new marker")


[-118.405771, 33.869024]


In [30]:
sample

## Extract all the band values of the area around the pin, aka sample

New or original pin

In [36]:
# Extract bands values
# Use reduceRegion to get a dictionary of means within sample
# Use getInfo() to get the dictionary from the server
DNs = s1_s2.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': sample,#.geometry(),
    'scale': 10,
    'maxPixels': 1e9
}).getInfo()

DNs


{'B11': 3184.6933115823817,
 'B12': 3216.2920065252856,
 'B2': 1757.0081566068518,
 'B3': 2279.0962479608484,
 'B4': 2640.035889070147,
 'B8': 2875.487765089723,
 'B8A': 2744.057096247961,
 'BSI': 0.10769141002653268,
 'NDWI': -0.1150085376084822,
 'VH': -27.597506885419296,
 'VV': -16.02240801542505,
 'mTGSI': 0.08973234113225462}

In [37]:
for b, band in enumerate(qm.USEFUL_BANDS):
    print('Band {}: {}'.format(band, DNs[band]))

Band B2: 1757.0081566068518 DN
Band B3: 2279.0962479608484 DN
Band B4: 2640.035889070147 DN
Band B8: 2875.487765089723 DN
Band B8A: 2744.057096247961 DN
Band B11: 3184.6933115823817 DN
Band B12: 3216.2920065252856 DN
Band VV: -16.02240801542505 DN
Band VH: -27.597506885419296 DN
Band mTGSI: 0.08973234113225462 DN
Band BSI: 0.10769141002653268 DN
Band NDWI: -0.1150085376084822 DN
